In [92]:
import numpy as np
from typing import Union, Callable
import pytest
import ipytest
from copy import deepcopy

import forward_propagation as fwd
from activation_functions import relu,sigmoid 
import test_functions

#%load_ext autoreload
%autoreload 2
np.random.seed(42)

## Tests

In [ ]:
def test_forward_nn_returns_correct_value():
    #given
    inputs = np.array([[1,2,5,4]])
    weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
    activation = sigmoid
    expected_output = np.array([[0.99899323, 1, 0.99945816]])
    #when
    output = fwd.forward_propagation(inputs,weights,activation)
    #then
    np.testing.assert_allclose(output, expected_output)
    
    
@pytest.mark.parametrize('activation', [
    (sigmoid),
    ([sigmoid]),
    ([[sigmoid,sigmoid,sigmoid]])
])
def test_forward_nn_returns_correct_value(activation):
    #given
    inputs = np.array([[1,2,5,4]])
    weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]

    expected_output = np.array([[0.99899323, 1, 0.99945816]])
    #when
    output = fwd.forward_propagation(inputs,weights,activation)
    #then
    np.testing.assert_allclose(output, expected_output)

ipytest.run("-qq")

In [ ]:
def test_vector_to_weights_returns_correct_value():
    # given
    vector = [0.2027827,-0.05644616,0.26441774,0.62177434,-0.09559271,-0.09558601,
              0.64471093,  0.31330392, -0.19166212,  0.22149921, -0.18918948,-0.19013338,
              0.09878068, -0.78109339, -0.70419476, -0.22955292, -0.41348657,0.12829094,
              -0.45401204, -0.70615185,  0.73282438, -0.11288815]
    network_structure = [5,3,1]
    expected_weights = [np.array([[ 0.2027827 , -0.05644616,  0.26441774,  0.62177434, -0.09559271,-0.09558601],
                               [ 0.64471093,  0.31330392, -0.19166212,  0.22149921, -0.18918948,-0.19013338],
                               [ 0.09878068, -0.78109339, -0.70419476, -0.22955292, -0.41348657,0.12829094]]),
                        np.array([[-0.45401204, -0.70615185,  0.73282438, -0.11288815]])]
    # when
    weights = fwd.vector_to_weights(vector,network_structure)
    # then
    for layer_weight,expected_layer_weight in zip(weights,expected_weights):
        np.testing.assert_allclose(layer_weight, expected_layer_weight)

def test_weights_to_vector_returns_correct_value():
    # given
    weights = [np.array([[ 0.2027827 , -0.05644616,  0.26441774,  0.62177434, -0.09559271,-0.09558601],
                               [ 0.64471093,  0.31330392, -0.19166212,  0.22149921, -0.18918948,-0.19013338],
                               [ 0.09878068, -0.78109339, -0.70419476, -0.22955292, -0.41348657,0.12829094]]),
                        np.array([[-0.45401204, -0.70615185,  0.73282438, -0.11288815]])]
    expected_vector = np.array([0.2027827,-0.05644616,0.26441774,0.62177434,-0.09559271,-0.09558601,
              0.64471093,  0.31330392, -0.19166212,  0.22149921, -0.18918948,-0.19013338,
              0.09878068, -0.78109339, -0.70419476, -0.22955292, -0.41348657,0.12829094,
              -0.45401204, -0.70615185,  0.73282438, -0.11288815])
    expected_structure = [5,3,1]
    # when
    vector, network_structure = fwd.weights_to_vector(weights)
    
    np.testing.assert_allclose(vector, expected_vector)
    np.testing.assert_allclose(network_structure, expected_structure)

ipytest.run("-qq")

## Multiple inputs 

In [18]:
inputs = np.array([[1,2,5,4]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
activation = sigmoid
fwd.forward_propagation(inputs,weights,activation)

[[ 6.9 ]
 [39.  ]
 [ 7.52]]
[6.9]
[39.]
[7.52]


array([[0.99899323, 1.        , 0.99945816]])

In [ ]:
inputs = np.array([[1,0.2,0.15,0.024]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
activation = sigmoid
fwd.forward_propagation(inputs,weights,activation)

In [28]:
inputs = np.array([[1,2,5,4],[1,0.2,0.15,0.024]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]
activation = sigmoid
fwd.forward_propagation(inputs,weights,activation)

[[ 6.9      0.139  ]
 [39.       2.77   ]
 [ 7.52     1.32872]]
[6.9   0.139]
[39.    2.77]
[7.52    1.32872]


array([[0.99899323, 1.        , 0.99945816],
       [0.53469416, 0.94103299, 0.79062883]])

In [ ]:
inputs = np.array([[1,2,5,4],[1,0.2,0.15,0.024]])
weights = [
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        ),
    np.array(
            [
                [1,0.2,0.5,1],
                [2,1,3,5]
            ]
        )
    ]
activation = sigmoid
fwd.forward_propagation(inputs,weights,activation)

### Creating a data set 
Create a data set of points sampled randomly from a function.

In [3]:

fun = test_functions.sphere
dim = 2
LB = [-5] * dim
UB = [5] * dim
ndata = 10

entry_data = np.random.uniform(low=LB,high=UB,size=(ndata,dim))
target = np.apply_along_axis(fun, 1, entry_data)

simulated_data = {
    "data": entry_data,
    "target": target
}

weights = fwd.create_weights([2,1])

In [9]:
weights

[array([[ 0.28677805, -0.07982693,  0.37394315]])]

In [8]:
entry_data

array([[-3.4398136 , -3.4400548 ],
       [-4.41916388,  3.66176146],
       [ 1.01115012,  2.08072578],
       [-4.79415506,  4.69909852],
       [ 3.32442641, -2.87660889],
       [-3.18175033, -3.1659549 ],
       [-1.95757757,  0.24756432],
       [-0.68054981, -2.0877086 ],
       [ 1.11852895, -3.60506139],
       [-2.07855351, -1.33638157]])

In [31]:
inputs

array([[1.   , 2.   , 5.   , 4.   ],
       [1.   , 0.2  , 0.15 , 0.024]])

In [30]:
weights

[array([[ 0.22213732, -0.06183368,  0.28965512,  0.68111966, -0.10471657],
        [-0.10470923,  0.70624544,  0.34320724, -0.20995533,  0.24264023]])]

In [26]:
[
        np.array(
            [
                [1,0.2,0.5,1,-1],
                [2,1,3,5,0],
                [0.2,0.1,0.6,0.78,1]
            ]
        )
    ]

[array([[ 1.  ,  0.2 ,  0.5 ,  1.  , -1.  ],
        [ 2.  ,  1.  ,  3.  ,  5.  ,  0.  ],
        [ 0.2 ,  0.1 ,  0.6 ,  0.78,  1.  ]])]

In [75]:
#inputs = np.array([[1,0.2,0.15,0.024]])
inputs = np.array([[1,1,1,1],[0.5,0.5,0.5,0.5],[1,1,1,1]])

In [93]:
weights = fwd.create_weights([4,2])
fwd.forward_propagation(inputs,weights,relu)

(2, 5)
(3, 5)
***
[[1.02636185 0.46082264 1.02636185]
 [0.97742836 0.61003429 0.97742836]]
(2, 3)
[1.02636185 0.97742836]
[0.46082264 0.61003429]
[1.02636185 0.97742836]
[[1.02636185 0.46082264 1.02636185]
 [0.97742836 0.61003429 0.97742836]]


array([[1.02636185, 0.97742836],
       [0.46082264, 0.61003429],
       [1.02636185, 0.97742836]])

In [66]:
d = np.array([2, 0, 3, -4, -2, 7, 9])
np.

[array([[ 0.22213732, -0.06183368,  0.28965512,  0.68111966, -0.10471657]])]

In [67]:
sum(weights[0].T)

array([1.02636185])